In [64]:
# from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, MinMaxScaler
# from pyspark.ml.clustering import KMeans
# from pyspark.sql import functions as F
# import sys
# from pyspark.sql import SparkSession
# import matplotlib.pyplot as plt
# from pyspark.sql import functions as F
# import seaborn as sns
# from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, BooleanType

In [65]:
# spark = SparkSession.builder \
#     .appName("Data Processing dans catalogue csv") \
#     .config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore:9083") \
#     .enableHiveSupport() \
#     .getOrCreate()
# print("Session Spark initialisée avec succès.")
# spark.sql("USE concessionnaire")
# catalogue_df = spark.sql("SELECT * FROM catalogue_co2_merge_processed")

In [66]:
# catalogue_df.show(10)
# catalogue_df.printSchema()


In [67]:

# Liste des colonnes catégorielles et numériques
# categorical_cols = ['marque', 'modele', 'couleur', 'longueur']
# numerical_cols = ['prix', 'unified_horse_power', 'rejets_co2', 'bonus_malus', 'nbplaces', 'nbportes']
# 
# # Étape 1 : Encodage des colonnes catégorielles
# indexers = [StringIndexer(inputCol=col, outputCol=f"{col}_index") for col in categorical_cols]
# for indexer in indexers:
#     catalogue_df = indexer.fit(catalogue_df).transform(catalogue_df)
# 
# # Vérifier les colonnes encodées
# encoded_cols = [f"{col}_index" for col in categorical_cols]
# 
# # Étape 2 : Préparer les colonnes numériques
# # Convertir toutes les colonnes numériques en double
# numerical_cols_full = numerical_cols + encoded_cols
# catalogue_df = catalogue_df.select(
#     *[F.col(c).cast('double').alias(c) for c in numerical_cols_full],
#     *[F.col(c) for c in ['marque', 'modele']]
# )
# 
# # Remplir les valeurs NULL par des valeurs par défaut
# catalogue_df = catalogue_df.fillna({col: 0 for col in numerical_cols_full})
# 
# # Étape 3 : Création du vecteur d'entrée
# assembler = VectorAssembler(inputCols=numerical_cols_full, outputCol='features_raw')
# assembled_df = assembler.transform(catalogue_df)
# 
# # Étape 4 : Normalisation des données
# scaler = MinMaxScaler(inputCol='features_raw', outputCol='features')
# scaled_model = scaler.fit(assembled_df)
# scaled_df = scaled_model.transform(assembled_df)

In [68]:
# # Étape 5 : Appliquer K-Means
# kmeans = KMeans(featuresCol='features', k=10, seed=1)  # k=5 comme point de départ
# model = kmeans.fit(scaled_df)
# 
# # Obtenir les prédictions de clusters
# clustered_df = model.transform(scaled_df)
# 
# # Résumé des clusters
# cluster_summary = clustered_df.groupBy('prediction').agg(
#     F.count('*').alias('count'),
#     *[F.mean(col).alias(f"avg_{col}") for col in numerical_cols]
# )
# cluster_summary.show()
# 
# # Ajouter les catégories en fonction des clusters
# def assign_category(prediction):
#     if prediction == 1:
#         return 'Citadines/compactes'
#     elif prediction == 2:
#         return 'Routières premium'
#     elif prediction == 4:
#         return 'Sportives premium'
#     else:
#         return 'Autres'
# 
# assign_category_udf = F.udf(assign_category, StringType())
# clustered_df = clustered_df.withColumn("categorie", assign_category_udf(F.col("prediction")))


In [69]:
# # Résumé des clusters
# cluster_summary = clustered_df.groupBy('prediction').agg(
#     F.count('*').alias('count'),
#     *[F.mean(col).alias(f"avg_{col}") for col in numerical_cols]
# )
# cluster_summary.show()
# 
# # Définir les catégories des clusters
# def assign_category(prediction):
#     if prediction == 1:
#         return 'Citadines/compactes'
#     elif prediction == 2:
#         return 'Routières premium'
#     elif prediction == 3:
#         return 'SUV'
#     elif prediction == 4:
#         return 'Sportives premium'
#     elif prediction == 5:
#         return 'Économiques'
#     else:
#         return 'Autres'
# 
# assign_category_udf = F.udf(assign_category, StringType())
# clustered_df = clustered_df.withColumn("categorie", assign_category_udf(F.col("prediction")))
# 
# # Afficher les données avec les catégories
# clustered_df.select('prediction', 'categorie', 'marque', 'modele', *numerical_cols).show(truncate=False)
# 
# # Réduction de dimensions avec PCA
# # Réduction de dimensions avec PCA
# pca = PCA(k=2, inputCol='features', outputCol='pca_features')
# pca_model = pca.fit(scaled_df)
# pca_result = pca_model.transform(clustered_df)  # Utilisez clustered_df, qui contient 'prediction'
# 
# # Convertir les résultats en Pandas pour visualisation
# pandas_df = pca_result.select('pca_features', 'prediction', 'marque', 'modele', 'categorie').toPandas()
# 
# # Séparer les composants PCA
# # Séparer les composants PCA
# pandas_df[['pca_x', 'pca_y']] = pandas_df['pca_features'].apply(lambda x: [x[0], x[1]]).to_list()
# 
# 
# # Visualisation des clusters
# plt.figure(figsize=(12, 8))
# sns.scatterplot(
#     data=pandas_df,
#     x='pca_x',
#     y='pca_y',
#     hue='categorie',
#     style='categorie',
#     palette='Set2',
#     s=100,
#     alpha=0.8
# )
# plt.title("Visualisation des Clusters avec PCA", fontsize=16)
# plt.xlabel("PCA Component 1", fontsize=12)
# plt.ylabel("PCA Component 2", fontsize=12)
# plt.legend(title="Catégories", fontsize=10, title_fontsize=12)
# plt.grid(True)
# plt.show()
# 
# 
# from pyspark.ml.evaluation import ClusteringEvaluator
# 
# # Tester différents nombres de clusters
# for k in range(3, 12):  # Par exemple, tester entre 3 et 7 clusters
#     kmeans = KMeans(featuresCol='features', k=k, seed=1)
#     model = kmeans.fit(scaled_df)
#     predictions = model.transform(scaled_df)
#     
#     # Évaluer les clusters avec le score silhouette
#     evaluator = ClusteringEvaluator(featuresCol='features')
#     silhouette = evaluator.evaluate(predictions)
#     print(f"Pour k={k}, score silhouette={silhouette}")
# 
# # Récupérer les centroïdes des clusters
# centroids = model.clusterCenters()
# 
# # Ajouter les centroïdes au graphique
# centroids_pca = pca_model.transform(
#     spark.createDataFrame(centroids, schema=["features"])
# ).toPandas()
# 
# centroids_pca[['pca_x', 'pca_y']] = centroids_pca['pca_features'].apply(lambda x: [x[0], x[1]]).to_list()
# 
# # Tracer les clusters et les centroïdes
# plt.figure(figsize=(12, 8))
# sns.scatterplot(
#     data=pandas_df,
#     x='pca_x',
#     y='pca_y',
#     hue='categorie',
#     style='categorie',
#     palette='Set2',
#     s=100,
#     alpha=0.8
# )
# 
# # Ajouter les centroïdes
# plt.scatter(
#     centroids_pca['pca_x'], 
#     centroids_pca['pca_y'], 
#     s=300, c='red', marker='X', label='Centroïdes'
# )
# 
# plt.title("Visualisation des Clusters avec PCA", fontsize=16)
# plt.xlabel("PCA Component 1", fontsize=12)
# plt.ylabel("PCA Component 2", fontsize=12)
# plt.legend(title="Catégories", fontsize=10, title_fontsize=12)
# plt.grid(True)
# plt.show()


In [70]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import when, col, lit
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml.clustering import KMeans

# Initialisation de la session Spark
spark = SparkSession.builder \
    .appName("Clustering Véhicules") \
    .enableHiveSupport() \
    .getOrCreate()
print("Session Spark initialisée avec succès.")
spark.sql("USE concessionnaire")

# Charger les données
catalogue_df = spark.sql("SELECT * FROM catalogue_co2_merge_processed")

# Étape 0 : Identification et correction des anomalies dans 'rejets_co2'

# 1. Identification des anomalies dans 'rejets_co2'
anomalies_df = catalogue_df.filter(
    (col('rejets_co2').isNull()) |
    (col('rejets_co2') == 0) |
    ((col('unified_horse_power') > 100) & (col('rejets_co2') < 50))
)

# 2. Calcul des moyennes nécessaires pour imputer les valeurs manquantes ou aberrantes

# a) Moyenne par modèle (en incluant les valeurs aberrantes pour assurer le calcul de la moyenne)
avg_co2_by_model = catalogue_df.filter(
    (col('rejets_co2').isNotNull()) &
    (col('rejets_co2') > 0)
).groupBy('marque', 'modele').agg(
    F.avg('rejets_co2').alias('avg_rejets_co2_model')
)

# b) Moyenne par marque
avg_co2_by_marque = catalogue_df.filter(
    (col('rejets_co2').isNotNull()) &
    (col('rejets_co2') > 0)
).groupBy('marque').agg(
    F.avg('rejets_co2').alias('avg_rejets_co2_marque')
)

# c) Moyenne générale
avg_co2_general = catalogue_df.filter(
    (col('rejets_co2').isNotNull()) &
    (col('rejets_co2') > 0)
).agg(
    F.avg('rejets_co2').alias('avg_rejets_co2_general')
).collect()[0]['avg_rejets_co2_general']

# 3. Imputation des valeurs manquantes ou aberrantes selon les consignes

# a) Joindre les moyennes au DataFrame des anomalies
anomalies_df = anomalies_df.join(avg_co2_by_model, on=['marque', 'modele'], how='left')
anomalies_df = anomalies_df.join(avg_co2_by_marque, on='marque', how='left')
anomalies_df = anomalies_df.withColumn('avg_rejets_co2_general', lit(avg_co2_general))

# b) Remplacer les valeurs de 'rejets_co2' selon les priorités
anomalies_df = anomalies_df.withColumn(
    'rejets_co2',
    when(
        col('avg_rejets_co2_model').isNotNull(),
        col('avg_rejets_co2_model')
    ).when(
        col('avg_rejets_co2_marque').isNotNull(),
        col('avg_rejets_co2_marque')
    ).otherwise(
        col('avg_rejets_co2_general')
    )
)

# c) Supprimer les colonnes temporaires de moyennes
anomalies_df = anomalies_df.drop('avg_rejets_co2_model', 'avg_rejets_co2_marque', 'avg_rejets_co2_general')

# 4. Mettre à jour le DataFrame initial avec les valeurs corrigées

# a) Exclure les anomalies du DataFrame initial
catalogue_non_anomalies = catalogue_df.join(
    anomalies_df.select('marque', 'modele', 'unified_horse_power', 'rejets_co2'),
    on=['marque', 'modele', 'unified_horse_power'],
    how='left_anti'
)

# b) Combiner les données corrigées avec le reste du DataFrame
catalogue_corrected_df = catalogue_non_anomalies.unionByName(anomalies_df)

# Vérification des corrections
# Afficher les valeurs de 'rejets_co2' pour les modèles Renault
catalogue_corrected_df.filter(col('marque') == 'renault').select(
    'marque', 'modele', 'rejets_co2', 'unified_horse_power'
).show()

# Étape 1 : Préparation pour le clustering

numerical_cols = ['prix', 'unified_horse_power', 'rejets_co2', 'bonus_malus', 'nbplaces', 'nbportes']

clustering_df = catalogue_corrected_df.select(
    *[col(c).cast('double').alias(c) for c in numerical_cols],
    'marque', 'modele'
).fillna({col: 0 for col in numerical_cols})

# Assemblage des variables en un vecteur de caractéristiques
assembler = VectorAssembler(inputCols=numerical_cols, outputCol='features_raw')
assembled_df = assembler.transform(clustering_df)

# Normalisation des données
scaler = MinMaxScaler(inputCol='features_raw', outputCol='features')
scaler_model = scaler.fit(assembled_df)
scaled_df = scaler_model.transform(assembled_df)

# Étape 2 : Clustering avec K-Means

k = 9
kmeans = KMeans(featuresCol='features', k=k, seed=1)
model = kmeans.fit(scaled_df)
clustered_df = model.transform(scaled_df)

# Étape 3 : Analyse des clusters

cluster_stats = clustered_df.groupBy('prediction').agg(
    F.count('*').alias('count'),
    F.mean('prix').alias('mean_prix'),
    F.mean('unified_horse_power').alias('mean_horse_power'),
    F.mean('rejets_co2').alias('mean_rejets_co2'),
    F.mean('bonus_malus').alias('mean_bonus_malus'),
    F.mean('nbplaces').alias('mean_nbplaces'),
    F.mean('nbportes').alias('mean_nbportes')
).orderBy('prediction')

cluster_stats.show(truncate=False)

# Étape 4 : Assignation des catégories

clustered_df = clustered_df.withColumn(
    "categorie",
    when(col("prediction") == 0, "Berlines hybrides haut de gamme") 
    .when(col("prediction") == 1, "Berlines de luxe")
    .when(col("prediction") == 2, "Voitures sportives haut de gamme")
    .when(col("prediction") == 3, "Citadines économiques")
    .when(col("prediction") == 4, "Familiales grandes places")
    .when(col("prediction") == 5, "Berlines intermédiaires thermiques")
    .when(col("prediction") == 6, "Micro-citadines électriques") 
    .when(col("prediction") == 7, "Citadines")
    .when(col("prediction") == 8, "Citadines compactes")
    .otherwise("Autres")
)

# Étape 5 : Vérification des catégories

categories = clustered_df.select("categorie").distinct().rdd.flatMap(lambda x: x).collect()
for cat in categories:
    print(f"\nExemples pour la catégorie : {cat}")
    clustered_df.filter(col("categorie") == cat).select(
        "marque", "modele", "prix", "unified_horse_power", "rejets_co2", "nbplaces", "nbportes"
    ).show(5, truncate=False)


Session Spark initialisée avec succès.
+-------+---------+-----------------+-------------------+
| marque|   modele|       rejets_co2|unified_horse_power|
+-------+---------+-----------------+-------------------+
|renault|   laguna|75.96444444444448|              170.0|
|renault|   megane|75.96444444444448|              135.0|
|renault|   laguna|75.96444444444448|              170.0|
|renault|   megane|75.96444444444448|              135.0|
|renault|   megane|75.96444444444448|              135.0|
|renault|   megane|75.96444444444448|              135.0|
|renault|   megane|75.96444444444448|              135.0|
|renault|   espace|75.96444444444448|              165.0|
|renault|vel satis|75.96444444444448|              245.0|
|renault|vel satis|75.96444444444448|              245.0|
|renault|   laguna|75.96444444444448|              170.0|
|renault|   megane|75.96444444444448|              135.0|
|renault|   laguna|75.96444444444448|              170.0|
|renault|   espace|75.96444444444